In [1]:
############### Import packages
import os, numpy as np, pandas as pd, time, glob, re, math, statsmodels.api as sm, patsy as ps
from tqdm import tqdm
from time import process_time
from datetime import datetime
from datetime import date
from openpyxl import load_workbook
from patsy import dmatrices

############### Set working directory to parent directory
if os.getcwd() != 'F:\\github\\narrative_conservatism\\code':
    os.chdir('F:\\github\\narrative_conservatism\\code')

############### Set pandas column printing constraint
pd.set_option('display.max_columns', None)

In [2]:
#############################################################################################
####### Merge IBES annual data (oen-year forecast) with CRSP_COMP_EDGAR quarterly data ######
#############################################################################################

########### Read IBES raw data file
ibes_cols = ['cusip8', 'cname', 'fpi', 'value', 'fpedats', 'anndats', 'actual', 'anndats_act']
ibes = pd.read_csv('..\\filings\\ibes.csv', usecols = ibes_cols)
print('number of cusip-fpedats-analyst: ' + str(ibes.shape[0]))

### Reorder and rename IBES columns
ibes = ibes[['cusip8', 'cname', 'fpedats', 'value', 'actual', 'anndats_act', 'anndats', 'fpi']]
ibes = ibes.rename(columns={'cusip8': 'cusip'})

### correct fpedats format
ibes['fpedats'] = ibes['fpedats'].str.replace('jan','01')
ibes['fpedats'] = ibes['fpedats'].str.replace('feb','02')
ibes['fpedats'] = ibes['fpedats'].str.replace('mar','03')
ibes['fpedats'] = ibes['fpedats'].str.replace('apr','04')
ibes['fpedats'] = ibes['fpedats'].str.replace('may','05')
ibes['fpedats'] = ibes['fpedats'].str.replace('jun','06')
ibes['fpedats'] = ibes['fpedats'].str.replace('jul','07')
ibes['fpedats'] = ibes['fpedats'].str.replace('aug','08')
ibes['fpedats'] = ibes['fpedats'].str.replace('sep','09')
ibes['fpedats'] = ibes['fpedats'].str.replace('oct','10')
ibes['fpedats'] = ibes['fpedats'].str.replace('nov','11')
ibes['fpedats'] = ibes['fpedats'].str.replace('dec','12')
ibes['fpedats'] = pd.to_datetime(ibes['fpedats'], format='%d%m%Y')

print(ibes['fpedats'].describe())

### Delete missing cusip8 and actual
del_cusip = ibes[ibes['cusip'].isnull()].shape[0]
ibes = ibes[ibes['cusip'].isnull() == False]
print('number of obs. that contain missing cusip: ' + str(del_cusip))

del_actual = ibes[ibes['actual'].isnull()].shape[0]
ibes = ibes[ibes['actual'].isnull() == False]
print('number of obs. that contain missing actual: ' + str(del_actual))

### aggregate by cusip-fpedats and get actual, median, afe and consensus from ibes raw data
ibes_css = ibes.groupby(['cusip', 'fpedats'])['actual'].median().to_frame()
ibes_css['median'] = ibes.groupby(['cusip', 'fpedats'])['value'].median().to_frame()
ibes_css['afe'] = ibes_css['actual'] - ibes_css['median']
ibes_css['consensus'] = ibes.groupby(['cusip', 'fpedats'])['value'].mean().to_frame()
ibes_css['leap_consensus'] = ibes_css.groupby(['cusip'])['consensus'].shift(-1)

### make a consensus forecast dataset by cusip-fpedats
ibes = ibes.loc[ibes.duplicated(subset=['cusip', 'fpedats']) == False, ['cusip','fpedats']]
ibes = ibes.join(ibes_css, on=['cusip', 'fpedats'])

### create merge date_key - keep %y only
ibes['date_key'] = ibes['fpedats'].astype(str).str[:-6].astype(int)

print('number of cusip-fpedats, dropping missing cusip and actual: ' + str(ibes.shape[0]))

number of cusip-fpedats-analyst: 9812071
count                 9812071
unique                    446
top       2015-12-31 00:00:00
freq                   426980
first     1981-12-31 00:00:00
last      2019-07-31 00:00:00
Name: fpedats, dtype: object
number of obs. that contain missing cusip: 60781
number of obs. that contain missing actual: 1305946
number of cusip-fpedats, dropping missing cusip and actual: 155539


In [3]:
##################### Read EDGAR_CRSP_COMP and creat merge date_key
crsp_comp_edgar = pd.read_csv('..\\filings\\crsp_comp_edgar_10-Q.csv')
crsp_comp_edgar['date_key'] = crsp_comp_edgar['fyearq'].astype(int)

##################### Inner merge EDGAR_CRSP_COMP and IBES, key not unique in both datasets
crsp_comp_edgar_ibes = pd.merge(crsp_comp_edgar, ibes, on = ['cusip', 'date_key'], how='inner', validate = 'm:m')
print('number of cusip-date after merging: ' + str(crsp_comp_edgar_ibes.shape[0]))

crsp_comp_edgar_ibes.sort_values(by = ['cusip', 'cquarter'])

number of cusip-date after merging: 110097


,cusip,cik,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,leap1_EARN,leap2_EARN,leap3_EARN,leap1_CFO,leap2_CFO,leap3_CFO,TACC,LAG_TA_REV,DSAR,PPE,SIC2,nw,nvocab,n_neg,n_pos,n_uctt,n_lit,n_cstr,n_modal_week,n_modal_moderate,n_modal_weak,n_negation,READ,tone,NW,TONE,TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus
107233,00030710,1606180,2015-03-31,0001564590-15-003247,"AAC Holdings, Inc.",178698.0,8060,2015-05-05,2015-03-31,2015-03-31,201503,2015.0,1.0,NV,NV,37027,A,182,99.585,59.186,1.340,2.038,26.089,17.742,42.823,0.749,NaN,188.983,145.952,101.145,97.474,21.781,21.374,4.440,4.357,71.628,24.284,30.580,30.9200,2.038,3.354,0.066935,0.234733,0.013963,0.0,-0.009017,NaN,-0.072387,0.029394,0.011661,0.001540,0.048258,-0.003657,0.001268,12.603,0.699826,-0.022871,0.484228,80,16287,1709,163,47,193,82,102,24,26,32,1,24.89,-0.007184,9.698184,-7.183643,35,0,6.493578,6.780106,0.196236,5.209486,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
107234,00030710,1606180,2015-06-30,0001564590-15-005942,"AAC Holdings, Inc.",178698.0,8060,2015-08-03,2015-06-30,2015-06-30,201506,2015.0,2.0,NV,NV,37027,A,273,97.666,45.021,1.676,5.555,28.972,30.118,53.784,0.303,NaN,210.265,188.983,109.824,101.145,21.816,21.781,5.227,4.440,70.641,71.628,43.560,30.5800,5.555,2.038,0.375746,0.011999,0.029394,0.0,0.018610,NaN,0.048258,0.011661,0.001540,0.001854,-0.003657,0.001268,-0.006056,-3.565,0.772302,0.007080,0.491055,80,21532,2044,305,85,301,182,138,32,33,91,13,27.31,-0.010821,9.977342,-10.821103,34,0,6.501384,6.585229,0.402512,5.613128,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
107235,00030710,1606180,2015-09-30,0001564590-15-010373,"AAC Holdings, Inc.",178698.0,8060,2015-11-10,2015-09-30,2015-09-30,201509,2015.0,3.0,NV,NV,37027,A,365,76.724,10.764,1.921,2.452,112.594,32.120,57.372,1.942,NaN,290.911,210.265,139.720,109.824,22.407,21.816,4.856,5.227,116.710,70.641,22.250,43.5600,2.452,5.555,-0.482160,0.121349,0.011661,0.0,-0.014758,0.009461,-0.003657,0.001540,0.001854,0.002768,0.001268,-0.006056,0.006473,3.221,0.898785,-0.034033,0.518812,80,21050,1947,232,68,214,140,108,27,32,39,7,29.19,-0.008124,9.954703,-8.123515,41,1,6.856783,8.652981,0.360821,5.902633,2015,2015-12-31,0.9700,0.7800,0.1900,0.753333,NaN
9664,00036020,824142,1996-09-30,0000106455-96-000015,AAON INC,21542.0,3585,1996-11-01,1996-09-30,1996-09-30,199609,1996.0,3.0,NV,NV,74107,A,2100,26.423,0.025,0.621,0.522,NaN,13.827,17.173,0.000,1.796,36.501,33.936,14.972,14.450,6.122,6.122,0.172,0.714,7.702,10.217,5.625,5.2500,0.522,0.475,0.038653,0.046985,0.015382,0.0,0.001385,0.005044,0.100660,0.018054,0.020130,0.020577,0.002548,-0.090078,0.024802,-2.894,0.888820,-0.029143,0.501680,35,1276,450,5,5,9,9,6,2,4,2,0,112.69,0.000000,7.152269,0.000000,32,0,3.470117,2.224256,0.322106,7.650169,1996,1996-12-31,0.0307,0.0702,-0.0395,0.071073,0.045367
25415,00036020,824142,1999-03-31,0001026608-99-000042,AAON INC,21542.0,3585,1999-05-14,1999-03-31,1999-03-31,199903,1999.0,1.0,NV,NV,74107,A,3012,31.559,0.369,0.732,1.764,NaN,15.500,30.036,NaN,4.266,50.827,50.506,26.223,24.411,6.227,6.219,0.757,0.757,9.104,10.980,11.000,9.3125,1.764,1.456,0.167819,0.136016,0.034927,0.0,0.006098,0.003367,-0.043361,0.047632,0.052270,0.047343,0.060263,0.034482,0.059992,3.954,1.011721,0.014573,0.647012,35,1660,602,21,7,26,13,8,5,7,4,0,59.16,-0.008434,7.415175,-8.433735,44,0,4.058967,2.372473,0.232388,8.010692,1999,1999-12-31,0.1308,0.0922,0.0386,0.102540,0.157771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [4]:
########################################################################################
############ Merge CRSP_COMP_EDGAR_IBES data with SEG data #############################
########################################################################################

########### Read SEG raw data file
seg_cols = ['gvkey', 'stype', 'sid', 'datadate', 'snms', 'cusip', 'cik']
seg = pd.read_csv('..\\filings\\compustat_seg.csv', usecols = seg_cols)
print('number of cusip-fpedats-analyst: ' + str(seg.shape[0]))

### Reorder and rename IBES columns
seg = seg[['gvkey', 'cik', 'cusip', 'datadate', 'stype', 'sid', 'snms']]
seg = seg.rename(columns={'datadate': 'date_comp'})

### parse date_comp format
seg['date_comp'] = pd.to_datetime(seg['date_comp'], format='%Y%m%d')

### aggregate by gvkey-date_comp-stype and get count of sid
seg_count = seg.groupby(['gvkey', 'date_comp', 'stype'])['sid'].count().to_frame()

### make a segment dataset by gvkey-date_comp-stype
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp', 'stype']) == False, ['gvkey', 'date_comp', 'stype']]
seg = seg.join(seg_count, on=['gvkey', 'date_comp', 'stype'])
seg_bus = seg[seg['stype']=='BUSSEG']
seg_bus = seg_bus.drop(columns=['stype'])
seg_bus = seg_bus.rename(columns={'sid': 'nseg_bus'})
seg_geo = seg[seg['stype']=='GEOSEG']
seg_geo = seg_geo.drop(columns=['stype'])
seg_geo = seg_geo.rename(columns={'sid': 'nseg_geo'})
seg = seg.loc[seg.duplicated(subset=['gvkey', 'date_comp']) == False, ['gvkey', 'date_comp']]
seg = pd.merge(seg, seg_bus, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = pd.merge(seg, seg_geo, on = ['gvkey', 'date_comp'], how='left', validate = '1:1')
seg = seg.sort_values(by=['gvkey', 'date_comp'])
seg.loc[seg['nseg_geo'].isnull(), 'nseg_geo'] = 1
seg.loc[seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
seg['nseg_bus'] = seg['nseg_bus'].astype(int)
seg['nseg_geo'] = seg['nseg_geo'].astype(int)

### create merge date_key - %y only
seg['date_key'] = seg['date_comp'].astype(str).str[:-6].astype(int)
seg = seg.drop(columns=['date_comp'])

print('number of gvkey-datadate in SEGMENT data: ' + str(seg.shape[0]))

number of cusip-fpedats-analyst: 452653
number of gvkey-datadate in SEGMENT data: 50876


In [5]:
##################### left merge EDGAR_CRSP_COMP_IBES and SEG, key not unique in both datasets
crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='left', validate = 'm:m')
print('Number of gvkey-quarter obs. after merging, left: '+ str(crsp_comp_edgar_ibes_seg.shape[0]))
# crsp_comp_edgar_ibes_seg_inner = pd.merge(crsp_comp_edgar_ibes, seg, on = ['gvkey', 'date_key'], how='inner', validate = 'm:m')
# print('Number of gvkey-quarter obs. after merging, inner: '+ str(crsp_comp_edgar_ibes_seg_inner.shape[0]))

##################### Fill missing segments in crsp_comp_edgar_ibes_seg_left with 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_bus'].isnull(), 'nseg_bus'] = 1
crsp_comp_edgar_ibes_seg.loc[crsp_comp_edgar_ibes_seg['nseg_geo'].isnull(), 'nseg_geo'] = 1

####### Drop duplicated gykey-cquarter
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg.duplicated(subset=['gvkey', 'cquarter']) == False]

Number of gvkey-quarter obs. after merging, left: 110116


In [6]:
########################################################################################
############################### Variable Creation ######################################
########################################################################################

################### CRSP_COMP_EDGAR_IBES_SEG_LEFT ######################################

######## AGE: log(1 + age from the first year the firm entered the CRSP dataset)
crsp_comp_edgar_ibes_seg['AGE'] = np.log(1 + crsp_comp_edgar_ibes_seg['age'])

######## BUSSEG: log(1 + number of business segments), or 1 if item is missing from Compustat; and
######## GEOSEG: log(1 + number of geographic segments), or 1 if item is missing from Compustat.
crsp_comp_edgar_ibes_seg['BUSSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_bus'])
crsp_comp_edgar_ibes_seg['GEOSEG'] = np.log(1 + crsp_comp_edgar_ibes_seg['nseg_geo'])

######## AFE, analyst forecast error, \
######## defined as I/B/E/S earnings per share minus the median of the most recent analysts’ forecasts, \
######## deflated by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AFE'] = crsp_comp_edgar_ibes_seg['afe']/crsp_comp_edgar_ibes_seg['prccq']

######## AF, analyst consensus forecast for one-year-ahead earnings per share, scaled by stock price per share at the end of the fiscal quarter
crsp_comp_edgar_ibes_seg['AF'] = crsp_comp_edgar_ibes_seg['leap_consensus']/crsp_comp_edgar_ibes_seg['prccq']

In [7]:
########################################################################################
########################## Variable Screening ##########################################
########################################################################################

########## Drop files (firm-quarter) that have positive/negative infinity DEARN
del_DEARN = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']==np.inf) | (crsp_comp_edgar_ibes_seg['DEARN']==-np.inf)].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[(crsp_comp_edgar_ibes_seg['DEARN']!=np.inf) & (crsp_comp_edgar_ibes_seg['DEARN']!=-np.inf)]
print('number of files that have positive/negative infinity DEARN: ' + str(del_DEARN))

### Drop missing AF
del_AF = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AF'].isnull() == False]
print('number of obs. that contain missing AF: ' + str(del_AF))

### Drop missing AFE
del_AFE = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['AFE'].isnull() == False]
print('number of obs. that contain missing AFE: ' + str(del_AFE))

### Drop missing EARN
del_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['EARN'].isnull() == False]
print('number of obs. that contain missing EARN: ' + str(del_EARN))

### Drop missing STD_EARN
del_STD_EARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['STD_EARN'].isnull() == False]
print('number of obs. that contain missing STD_EARN: ' + str(del_STD_EARN))

### Drop missing EARN
del_DEARN = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull()].shape[0]
crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DEARN'].isnull() == False]
print('number of obs. that contain missing DEARN: ' + str(del_DEARN))

############## Winsorize ##############################
###### Define a function that winsorize a variable at 1% and 99% 
def winsorize (df, colnames):
    for col in colnames:
        varq01 = df[col].quantile(.01)
        varq99 = df[col].quantile(.99)
        df[col] = df[col].clip(varq01, varq99)
    return df

############## Winsorize TABLE 1 (line 1) and TABLE 4 (line 2) variables
crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, \
                                          ['AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', \
                                          'CFO', 'leap1_EARN', 'leap2_EARN', 'leap3_EARN', 'leap1_CFO', 'leap2_CFO', 'leap3_CFO', 'READ'])

############## Inspect sample size after variable screening
print('Number of firm-quarters after variable screening: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

number of files that have positive/negative infinity DEARN: 0
number of obs. that contain missing AF: 15230
number of obs. that contain missing AFE: 0
number of obs. that contain missing EARN: 64
number of obs. that contain missing STD_EARN: 3162
number of obs. that contain missing DEARN: 0
Number of firm-quarters after variable screening: 91607


In [8]:
######## Create ABTONE: residual from TONE regression 
y, X = ps.dmatrices('TONE ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
                    data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
model = sm.OLS(y, X)
res = model.fit()
# res.summary()
crsp_comp_edgar_ibes_seg['ABTONE'] = res.resid

######## Create abtone for REPLICATION: residual from tone regression 
y, X = ps.dmatrices('tone ~ 1 + EARN + RET + SIZE + MTB + STD_RET + STD_EARN + AGE + BUSSEG + GEOSEG + LOSS + DEARN + AFE + AF', \
                    data = crsp_comp_edgar_ibes_seg, return_type = 'dataframe')
model = sm.OLS(y, X)
res = model.fit()
# res.summary()
crsp_comp_edgar_ibes_seg['abtone'] = res.resid

############## Save merged CRSP_COMP_EDGAR_IBES_SEG to csv file
crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_10-Q.csv', index = 0)

crsp_comp_edgar_ibes_seg

,cusip,cik,rp,accnum,name,gvkey,SIC,fd,date_crsp,date_comp,cquarter,fyearq,fqtr,incorp,state,addzip,costat,age,actq,cheq,dpq,ibq,intanq,lctq,revtq,txditcq,xsgaq,atq,lag_atq,ceqq,lag_ceqq,cshoq,lag_cshoq,dlcq,lag_dlcq,dlttq,lag_dlttq,prccq,lag_prccq,ibq.1,lag_ibq,RET,STD_RET,EARN,LOSS,DEARN,STD_EARN,CFO,leap1_EARN,leap2_EARN,leap3_EARN,leap1_CFO,leap2_CFO,leap3_CFO,TACC,LAG_TA_REV,DSAR,PPE,SIC2,nw,nvocab,n_neg,n_pos,n_uctt,n_lit,n_cstr,n_modal_week,n_modal_moderate,n_modal_weak,n_negation,READ,tone,NW,TONE,TLAG,NEG,SIZE,MTB,LEV,AGE,date_key,fpedats,actual,median,afe,consensus,leap_consensus,nseg_bus,nseg_geo,BUSSEG,GEOSEG,AFE,AF,ABTONE,abtone
0,54626810,60512,1993-06-30,0000060512-94-000005,LOUISIANA LAND & EXPLORATION CO,6819.0,1311,1993-08-13,1993-06-30,1993-06-30,199306,1993.0,2.0,MD,MD,70112,I,11324,193.800,64.300,27.300,5.600,NaN,208.200,189.300,136.100,23.900,1278.000,1203.400,424.200,421.300,28.729,28.647,86.500,0.000,356.300,379.500,42.500,45.250,5.600,2.700,-0.059108,0.054464,0.004653,0.0,0.002410,0.002744,0.031660,-0.001408,0.003730,0.003372,0.037089,0.041509,-0.075597,-32.500,1.004737,-0.000249,1.996925,13,3187,743,66,6,23,43,6,9,6,6,0,262.5148,-0.018826,8.067149,-18.826483,44,1,7.167251,3.076850,0.315356,9.334768,1993,1993-12-31,0.52,1.00,-0.48,0.990359,0.714757,1.0,1.0,0.693147,0.693147,-0.011294,0.016818,-11.282795,-0.011283
1,54626810,60512,1993-09-30,0000060512-94-000007,LOUISIANA LAND & EXPLORATION CO,6819.0,1311,1993-11-10,1993-09-30,1993-09-30,199309,1993.0,3.0,MD,MD,70112,I,11416,180.200,9.400,28.200,-1.800,NaN,195.100,187.900,124.800,23.700,1662.300,1278.000,417.100,424.200,33.137,28.729,0.000,86.500,747.700,356.300,44.625,42.500,-1.800,5.600,0.025969,0.094378,-0.001408,1.0,-0.005790,0.004136,0.037089,0.003730,0.003372,0.000335,0.041509,-0.075597,0.026837,-49.200,0.941628,-0.031768,2.206729,13,3965,789,71,13,23,56,3,16,8,4,0,262.5148,-0.014628,8.285513,-14.627995,41,0,7.107411,2.878318,0.346479,9.342859,1993,1993-12-31,0.52,1.00,-0.48,0.990359,0.714757,1.0,1.0,0.693147,0.693147,-0.010756,0.016017,-3.613834,-0.003614
2,08750910,11860,1993-09-30,0000011860-94-000005,BETHLEHEM STEEL CORP /DE/,2189.0,3312,1993-11-12,1993-09-30,1993-09-30,199309,1993.0,3.0,DE,DE,18016-7699,I,24715,1074.900,232.500,69.100,30.700,NaN,908.400,1055.300,0.000,38.400,5168.098,5463.199,580.800,958.900,91.307,91.025,100.300,88.400,728.500,650.200,14.375,18.750,30.700,-13.600,-0.270146,0.119124,0.005619,0.0,0.008109,0.007031,0.007871,-0.046942,0.002195,0.004469,0.025212,-0.013205,0.004332,-12.300,1.004723,-0.008823,1.315529,33,4032,940,81,38,23,62,32,19,12,5,0,94.7700,-0.010665,8.302266,-10.664683,43,1,7.442328,1.779871,0.135196,10.115206,1993,1993-12-31,0.41,0.30,0.11,0.610301,1.489195,1.0,1.0,0.693147,0.693147,0.007652,0.103596,-2.186519,-0.002187
3,91335310,101929,1993-11-30,0000101929-94-000002,UNIVAR CORP,11003.0,5160,1994-01-13,1993-11-30,1993-11-30,199311,1993.0,3.0,DE,DE,98033,I,9041,372.727,18.416,6.890,0.458,NaN,281.853,430.299,NaN,60.117,654.434,660.336,160.280,161.722,19.643,19.635,42.298,47.584,150.563,152.012,11.000,13.250,0.458,1.957,-0.170798,0.066070,0.000694,0.0,-0.002270,0.002216,0.037449,-0.000348,0.002294,-0.026588,-0.015458,-0.057851,0.009150,-24.271,1.084569,-0.049554,NaN,51,2795,638,20,8,12,62,13,2,5,3,0,82.1400,-0.004293,7.935945,-4.293381,44,1,5.561311,1.608710,0.302264,9.109636,1993,1993-02-28,0.27,0.58,-0.31,0.636296,0.586216,1.0,1.0,0.693147,0.693147,-0.028182,0.053292,2.512238,0.002512
4,91345610,102037,1993-12-31,0000950118-94-000014,UNIVERSAL CORP /VA/,11017.0,5150,1994-02-11,1993-12-31,1993-12-31,199312,1994.0,2.0,VA,VA,23235,A,24168,1285.950,66.170,NaN,20.210,NaN,947.603,866.774,28.519,73.103,1783.210,1758.267,406.682,393.913,35.634,35.631,578.773,577.178,294.965,302.786,25.625,22.375,20.210,18.459,0.133202,0.057684,0.011494,0.0,0.000996,0.008327,0.003720,0.005239,-0.003138,0.002387,0.017610,0.089023,-0.102212,13.669,0.888372,0.068531,0.303600,51,2094,654,20,26,24,18,19,3,3,1,0,47.0800,0.002865,7.647309,

In [9]:
# ####################################################################################################################
# ############# Create COMP_DA: if not replicating TABLE 4, DO NOT run because cause large sample reduction ##########
# ####################################################################################################################

# ### Drop missing or infinite LAG_TA_REV, DSAR and PPE
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['LAG_TA_REV'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['DSAR'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['DSAR'] != -np.inf)]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg.loc[(crsp_comp_edgar_ibes_seg['PPE'] != np.inf) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'].isnull() == False) & \
#                                                         (crsp_comp_edgar_ibes_seg['PPE'] != -np.inf)]
# print('number of obs. after deleting missing or infinite LAG_TA_REV, DSAR and PPE: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Winsorize DA variables
# crsp_comp_edgar_ibes_seg = winsorize(crsp_comp_edgar_ibes_seg, ['TACC', 'LAG_TA_REV', 'DSAR', 'PPE'])

# ######## Create DA: residual from TACC regression by each two-digit SIC-year
# def TACC_reg(data):
#     try:
#         y, X = ps.dmatrices('TACC ~ 1 + LAG_TA_REV + DSAR + PPE', data = data, return_type = 'dataframe')
#         model = sm.OLS(y, X)
#         res = model.fit()
#         data['DA'] = res.resid
#         return data
#     except:
#         pass

# crsp_comp_edgar_ibes_seg1 = crsp_comp_edgar_ibes_seg.groupby(['SIC2','fyearq']).apply(TACC_reg)
# # comp_DA['DA'].describe()

# ####### Join DA to COMP
# crsp_comp_edgar_ibes_seg = pd.merge(crsp_comp_edgar_ibes_seg, crsp_comp_edgar_ibes_seg1[['cik', 'rp', 'DA']], on = ['cik', 'rp'], how='left', validate = '1:1')

# crsp_comp_edgar_ibes_seg

# ############## Dropping missing DA to replicate Huang et al. 2014 TABLE 4, \
# del_DA = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull()].shape[0]
# crsp_comp_edgar_ibes_seg = crsp_comp_edgar_ibes_seg[crsp_comp_edgar_ibes_seg['DA'].isnull() == False]
# print('number of obs. that contain missing DA: ' + str(del_DA))

# ############## Inspect sample size after variable screening
# print('Number of firm-quarters after dropping missing DA: ' + str(crsp_comp_edgar_ibes_seg.shape[0]))

# ############## Save merged ID_CRSP_COMP_TEXT to csv file
# crsp_comp_edgar_ibes_seg.to_csv('..\\filings\\crsp_comp_edgar_ibes_seg_DA_10-Q.csv', index = 0)

In [16]:
########################################################################################
########### Table 2 - Panel A: Summary Statistics and Correlation Matrix ###############
########################################################################################

############# Table 2 Panel A: Summary statistics for selected variables
######### Variable groups:
# 1st line: textual variables, generally consistent with LM's summary statistics
# 2nd line: fundamental variables (main)
# 3rd line: abtone
selected_vars = crsp_comp_edgar_ibes_seg[['NW','nw', 'TONE','TLAG', 'READ', \
                                          'RET', 'NEG', 'SIZE', 'MTB', 'LEV', \
                                          'AF', 'AFE', 'BUSSEG','GEOSEG','AGE','EARN', 'DEARN', 'STD_EARN', 'STD_RET', 'LOSS', 'ABTONE',\
                                        # 'DA' 
                                        ]]

T2PA = selected_vars.describe().transpose() 

############ count number of unique firms
print('Number of unique firms in final 10-Q sample: ' + str(crsp_comp_edgar_ibes_seg['cik'].unique().shape[0]))

############# Summary statistics for all raw and processed variables
full_summary = crsp_comp_edgar_ibes_seg.describe().transpose()

############# Save T3PA
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PA.to_excel(writer, sheet_name='T2PA_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PA.to_excel(table_path, sheet_name='T2PA_raw', float_format="%.4f")

T2PA

Number of unique firms in final 10-Q sample: 5250


,count,mean,std,min,25%,50%,75%,max
NW,91607.0,9.020044e+00,0.756985,7.120444,8.505627,9.085910,9.546670,13.544145
nw,91607.0,1.093746e+04,10203.945299,1236.000000,4941.500000,8829.000000,13997.000000,762337.000000
TONE,91607.0,-8.920562e+00,7.235631,-63.578565,-13.127248,-7.875232,-3.865979,24.214907
TLAG,91607.0,3.902035e+01,6.249347,0.000000,36.000000,40.000000,44.000000,52.000000
READ,91607.0,3.816092e+01,42.159746,14.580000,17.840000,20.210000,39.660000,262.514800
RET,91607.0,1.827953e-02,0.253079,-1.578704,-0.112612,0.007313,0.129929,4.849226
NEG,91607.0,4.828998e-01,0.499710,0.000000,0.000000,0.000000,1.000000,1.000000
SIZE,91607.0,6.447013e+00,1.776252,2.001720,5.175091,6.317309,7.562823,11.206075
MTB,91607.0,3.515265e+00,4.008908,0.287839,1.485162,2.343014,3.901627,30.902199
LEV,91607.0,1.924492e-01,0.182163,0.000000,0.010646,0.162396,0.315176,0.724243


In [12]:
# full_summary

In [13]:
############# Table 3 Panel B: Correlation matrix for selected variables
######### pearson correlation
T2PB_pearson = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='pearson')

# T3PB_pearson

In [14]:
######### spearman correlation
T2PB_spearman = selected_vars.loc[:, (selected_vars.columns != 'nw') & \
                                 (selected_vars.columns != 'BUSSEG') & \
                                 (selected_vars.columns != 'GEOSEG') & \
                                 (selected_vars.columns != 'LOSS')].corr(method='spearman')

# T3PB_spearman

In [15]:
######### Combine two correlation matrices. right-up matrix: pearson; left-down matrix: spearman 
for row in list(range(0, len(T2PB_spearman.index))):
    T2PB_spearman.iloc[row, row+1:] = T2PB_pearson.iloc[row, row+1:]
    
##### Save T2PB
table_path = '..\\output\\Tables.xlsx'
if os.path.exists(table_path) == True:
    book = load_workbook(table_path)
    writer = pd.ExcelWriter(table_path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    T2PB_spearman.to_excel(writer, sheet_name='T2PB_raw', float_format="%.4f")

    writer.save()
    writer.close()
    
else:
    T2PB_spearman.to_excel(table_path, sheet_name='T2PB_raw', float_format="%.4f")

T2PB_spearman

,NW,TONE,TLAG,READ,RET,NEG,SIZE,MTB,LEV,AF,AFE,AGE,EARN,DEARN,STD_EARN,STD_RET,ABTONE
NW,1.000000,-0.455693,-0.192174,-0.082593,-0.006464,0.002245,0.255384,0.057453,0.036042,-0.067753,0.010805,-0.040214,-0.116293,0.001621,0.091214,-0.030267,-3.849690e-01
TONE,-0.481466,1.000000,0.015539,0.086127,0.020357,-0.020807,-0.061743,-0.013969,0.071639,0.071545,0.101819,0.059052,0.157343,-0.002893,-0.148004,-0.088735,9.561757e-01
TLAG,-0.263261,0.021266,1.000000,0.047966,-0.022210,0.034012,-0.330778,-0.021750,0.009432,-0.092250,-0.127343,-0.228377,-0.136763,-0.004550,0.121081,0.189194,1.694234e-02
READ,-0.251936,0.169254,0.124632,1.000000,-0.016437,0.015696,-0.014264,-0.035026,0.062856,0.045042,0.002045,0.088265,0.059172,0.002208,-0.047370,-0.050854,6.109392e-02
RET,-0.006911,0.028302,-0.032300,-0.028803,1.000000,-0.684122,-0.064464,-0.025810,0.001968,-0.018165,0.154627,0.001703,0.063448,0.036141,0.010768,0.266061,-1.571125e-16
NEG,0.003372,-0.023506,0.032701,0.028259,-0.865519,1.000000,0.000098,0.012509,-0.001902,0.015156,-0.124063,-0.017917,-0.071295,-0.018664,0.015691,-0.117616,4.221366e-04
SIZE,0.263978,-0.046533,-0.332576,-0.077923,-0.024161,-0.001004,1.000000,0.233536,0.099647,0.077190,0.269668,0.343981,0.258838,-0.023619,-0.197663,-0.310121,6.531052e-18
MTB,0.046309,0.039931,-0.042137,-0.025544,-0.054681,0.032940,0.382007,1.000000,0.045733,-0.156078,0.120161,-0.088334,-0.041167,0.022203,0.158513,0.035832,-5.611197e-16
LEV,0.013848,0.076414,-0.000416,0.075072,0.003009,-0.004073,0.142834,-0.111068,1.000000,0.166881,-0.068197,0.101104,0.039389,0.033884,-0.124433,-0.072001,6.996328e-02
AF,-0.018333,0.061766,-0.124698,0.035299,-0.086554,0.071511,0.025571,-0.299097,0.250819,1.000000,0.057264,0.202124,0.472273,0.015995,-0.256473,-0.145248,-2.413300e-16
